In [ ]:

import pandas as pd

def load_swc_to_graph(swc_file):
    df = pd.read_csv(
        swc_file,
        comment='#',
        delim_whitespace=True,
        names=["id", "type", "x", "y", "z", "radius", "parent"]
    )

    G = nx.Graph()
    node_positions = {}

    for _, row in df.iterrows():
        node_id = int(row["id"])
        parent_id = int(row["parent"])
        coord = np.array([row["x"], row["y"], row["z"]])
        node_positions[node_id] = coord
        G.add_node(node_id)
        if parent_id != -1:
            parent_coord = node_positions[parent_id]
            dist = np.linalg.norm(coord - parent_coord)
            G.add_edge(parent_id, node_id, weight=dist)

    return G, node_positions


def compute_synaptic_embeddings(
    client,
    segment_id,
    nucleus_id,
    skel_path,
    k=20,
    alpha=1.0,
    synapse_table_name='synapse_table',
    output_csv_path=None,
):
    """
    Compute synaptic embeddings for a given neuron based on its morphology and synapses.

    Parameters:
    - client: Initialized CAVEclient object
    - segment_id: Root ID of the neuron
    - nucleus_id: Nucleus ID (for SWC filename)
    - skel_path: Path to the skeleton SWC file
    - k: Number of Laplacian modes (default 20)
    - alpha: Fractional exponent (currently unused, placeholder for future extension)
    - synapse_table_name: Name of the synapse table
    - output_csv_path: If given, save the embeddings to this CSV file

    Returns:
    - DataFrame of synapse embeddings
    """
    import pandas as pd
    import numpy as np
    import networkx as nx
    from scipy.sparse.linalg import eigsh
    from sklearn.neighbors import KDTree
    from neuron_morphology.swc_io import read_swc

    # Step 1: Load skeleton and build graph
    skel_file = f"{skel_path}/{segment_id}.swc"

    G, node_positions = load_swc_to_graph(skel_file)

    for src, tgt in edges:
        dist = np.linalg.norm(node_positions[src] - node_positions[tgt])
        G.add_edge(src, tgt, weight=dist)

    # Step 2: Compute Laplacian and eigenvectors
    L = nx.laplacian_matrix(G, weight='weight')
    eigenvalues, eigenvectors = eigsh(L, k=k, which='SM')

    # Step 3: Query only relevant synapses
    synapse_df = client.materialize.query_table(
        synapse_table_name,
        split_positions=True,
        filter_in_dict={'post_pt_root_id': [segment_id]}
    )

    if synapse_df.empty:
        print(f"No synapses found for segment ID {segment_id}.")
        return pd.DataFrame()

    # Step 4: Map synapses to nearest nodes
    skeleton_coords = np.array(list(node_positions.values()))
    tree = KDTree(skeleton_coords)
    synapse_coords = synapse_df[['x', 'y', 'z']].values
    distances, indices = tree.query(synapse_coords, k=1)
    nearest_nodes = [list(node_positions.keys())[idx[0]] for idx in indices]

    # Step 5: Embed synapses
    embeddings = []
    for node_id in nearest_nodes:
        node_index = list(G.nodes).index(node_id)
        embedding = eigenvectors[node_index] / eigenvalues
        embeddings.append(embedding)

    embedding_df = pd.DataFrame(embeddings, columns=[f'mode_{i+1}' for i in range(k)])
    embedding_df['synapse_id'] = synapse_df['id'].values

    if output_csv_path:
        embedding_df.to_csv(output_csv_path, index=False)

    return embedding_df

from caveclient import CAVEclient

from caveclient import CAVEclient
from meshparty import skeleton_io
import os

# Initialize CAVEclient
client = CAVEclient('minnie65_public')

# Segment ID of the neuron
segment_id = 864691135122603047

# Directory to save
save_dir = "/data"
os.makedirs(save_dir, exist_ok=True)

# Pull the skeleton
skeleton = client.materialize.get_skeleton(segment_id)

# Save as SWC file
swc_path = os.path.join(save_dir, f"{segment_id}.swc")
skeleton_io.write_swc(skeleton, swc_path)

print(f"Saved SWC to: {swc_path}")


client = CAVEclient('minnie65_public')
segment_id = 864691135122603047
nucleus_id = 292685
skel_path = "/data"

df = compute_synaptic_embeddings(
    client,
    segment_id=segment_id,
    nucleus_id=nucleus_id,
    skel_path=skel_path,
    k=20,
    output_csv_path="synaptic_embedding.csv"
)


ModuleNotFoundError: No module named 'meshparty'

In [9]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
import os

from caveclient import CAVEclient
import pandas as pd
import numpy as np
import os

def save_skeleton_as_swc(client, segment_id, save_path):
    from caveclient import CAVEclient
    client = CAVEclient("minnie65_public")
    skel = client.chunkedgraph.get_skeleton(segment_id)

    vertices = skel.vertices
    edges = skel.edges

    node_ids = np.arange(len(vertices))
    parent_map = {child: parent for parent, child in edges}

    swc_rows = []
    for i, (x, y, z) in enumerate(vertices):
        swc_id = node_ids[i]
        swc_type = 3  # dendrite
        radius = 1.0
        parent = parent_map.get(swc_id, -1)
        swc_rows.append([swc_id, swc_type, x, y, z, radius, parent])

    swc_df = pd.DataFrame(swc_rows, columns=["id", "type", "x", "y", "z", "radius", "parent"])
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    swc_df.to_csv



# Usage
client = CAVEclient("minnie65_public")
segment_id = 864691135122603047
save_path = "/data/864691135122603047.swc"

save_skeleton_as_swc(client, segment_id, save_path)


AttributeError: 'ChunkedGraphClient' object has no attribute 'get_skeleton'

In [11]:
from caveclient import CAVEclient
import numpy as np
import pandas as pd

client = CAVEclient("minnie65_public")

# specify the materialization version, for consistency across time",
client.version = 1300

# Example: pyramidal cell in v1300
example_cell_id = 864691135572530981

sk_df = client.skeleton.get_skeleton(example_cell_id, output_format='swc')

2025-04-17 01:48:58,373 root         INFO     get_versions()
2025-04-17 01:48:58,374 root         INFO     endpoint: {skeleton_server_address}/skeletoncache/api/versions
2025-04-17 01:48:58,374 root         INFO     url: https://minnie.microns-daf.com/skeletoncache/api/versions
2025-04-17 01:48:58,578 root         INFO     response: <Response [200]>
2025-04-17 01:48:58,579 root         INFO     versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
2025-04-17 01:48:58,579 root         INFO     SkeletonService version: 0.18.1
2025-04-17 01:49:00,080 root         INFO     get_skeleton() response contains content of size 108925 bytes


In [10]:
from meshparty import meshwork
mw = meshwork.load_meshwork("s3://bossdb-open-data/iarpa_microns/minnie/minnie65/skeletons/v661/meshworks/864691135122603047.h5")

ModuleNotFoundError: No module named 'meshparty'